# Shockwave Traffic Jam

## Brief
L'addestramento consiste nel miglioramento dei parametri del modello a seguito di un batch di simulazioni.

### Make the dataset
Ogni dato del dataset è ottenuto stabilendo casualmente:
1. Lunghezza della strada
2. Numero di veicoli
3. Parametri per **OVM**
4. Parametri per **FTL**
5. Bilanciamento tra **OVM** e **FTL**
Si crea così un dato senza addensamenti
Le velocità sono ottenute in accordo con il modello **OVM**.

### Data Augmentation
Si aumentano il numero di dati in modo casuale proponendo una perturbazione delle velocità e un tempo di simulazione iniziale prima di attivare l'apprendimento.

Ad ogni chiamata del training quindi ci sarà l'aggiunta di rumore.

### Training
Si addestra la rete con due sistemi di premi:
1. feedback ad ogni tempo $\tau$ che suggeriscono alla rete quanto il suo attuale andamento è gradevole (accelerazioni contenute):
   1. Si tratta di un fine-tuning degli ultimi layer (più istintivi).
   2. Il simulatore prenderà traccia dell'accelerazione più forte (in modulo).
2. premio finale al tempo $T$ che indica alla rete lo spazio percorso complessivo:
   1. Si aggiorna il ragionamento e la memoria della rete, si tratta di un tuning dei layer più superficiali.
   2. Il simulatore prenderà traccia dello spazio percorso totale dal veicolo autonomo.

## The model
Il modello non ha accesso completo a tutta la strada, tuttavia può ricordare quello che vede e quello che ha fatto.

In questo modo in una strada lineare semplicemente il modello assocerà ciò che vede ad un'esperienza antecedente e anche se non è vero che quanto farà lo aiuterà, agirà comunque come se fosse per il bene proprio.

Per questa ragione usiamo delle RNN per l'addestramento di alto livello, semplicemente la rete memorizzerà la qualità della strategia nelle RNN

L'interpretazione delle strategie sarà fatta a livelli più profondi che ne stimano l'applicazione e che saranno addestrati con fine-tuning

# Make the dataset

Il dataset generato è solo una radice di costruzione del vero dataset usato per l'addestramento.

Generiamo quindi questa radice che prende in input:
- range di valori per la densità
- range di valori per $d_0$ richiesto da OVM
- range di valori per $\Delta$

La radice propone una distribuzione uniforme dei veicoli in uno stato stazionario.

In [8]:
# MAKE DATASET ROOT
from source import ShockwaveTrafficJam
import torch
import os

dataset_path = 'data/db/TRAFFIC'
os.makedirs(dataset_path, exist_ok=True)

ShockwaveTrafficJam.dataset.make_traffic(8_192, torch.device('cuda'), 'data/db/TRAFFIC')

Il dataset è quindi un tensore avente 3 colonne indicanti:
- densità
- d0_OVM
- Delta

In [9]:
# LOAD DATASET ROOT
from source import ShockwaveTrafficJam

dataset = ShockwaveTrafficJam.dataset.Dataset(dataset_path, root=True).to(torch.device('cuda'))

Il dataloader carica la radice e viene usato per produrre facilmente i batch.

Ogni batch presenta un'informazione completa e parallelizzata:
- posizioni dei veicoli : tensore di shape (batch_size, n_vehicles)
- velocità dei veicoli : tensore di shape (batch_size, n_vehicles)
- len_road, d0_OVM, Delta, Vmax, tau, d0_FTL, gamma, beta : tensore di shape (batch_size, 8)

In particolare sarà necessario passare al dataloader:
- range per il numero di veicoli : usato per generare il numero di veicoli del batch
- range per Vmax, tau, d0_FTL, gamma, beta
  
Sarà inoltre aggiunto un trasformatore dei dati che fungerà da data augmentation.

In [10]:
# CREATE DATALOADER ROOT
from source import ShockwaveTrafficJam

transformer = ShockwaveTrafficJam.transforms.Compose([
    ShockwaveTrafficJam.transforms.RandomDropout(0.1),                # 10% of the data will be dropped
    ShockwaveTrafficJam.transforms.RandomNoise(1.0, 2.0),             # perturbation of position and speed
    ShockwaveTrafficJam.transforms.Simulation(1.0/30.0, 14_400),       # simulate the traffic jam (8 min) with 30FPS
])

dataloader = ShockwaveTrafficJam.dataloader.DataLoader(dataset, batch_size=256, shuffle=True, transform=transformer, root=True).to(torch.device('cuda'))

Uso il dataloader per creare il dataset completo che sarà poi usato per l'addestramento

In [11]:
# make the dataset
from tqdm.notebook import tqdm

for i, sim in tqdm(enumerate(dataloader), total=len(dataloader), leave=False):
    sim = sim.to(torch.device('cpu'))
    sim.save(i, 'data/db/TRAFFIC')

  0%|          | 0/32 [00:00<?, ?it/s]

# Open dataset

E' possibile aprire direttamente il nuovo dataset con tutti gli scenari costruiti.

Ogni file rappresenta una parte del dataset, il dataloader carica i file in RAM e poi li smaltisce per l'addestramento.



In [2]:
# LOAD the dataset
import torch
from source import ShockwaveTrafficJam

# libero la gpu da ogni variabile di torch
torch.cuda.empty_cache()

dataset = ShockwaveTrafficJam.dataset.Dataset('data/db/TRAFFIC', root=False)
dataloader = ShockwaveTrafficJam.dataloader.DataLoader(dataset, batch_size=256)

In [3]:
# visualize a single data
dataloader = iter(dataloader)
sim = next(dataloader)
sim.visual()

## About the model

La rete fa uso di una `CNN` applicata alle misurazioni effettuate a $30$FPS, viene fornita una serie temporale di $316$ step con $2$ canali (circa $10$ secondi):
- distanza relativa
- velocità relativa

La sequenza viene poi semplificata ottenendo $64$ sequenze a $16$ step (con un frame rate simile al tempo di reazione umano).

Dopodiché viene passato il risultato attraverso un `trasformatore` che ne deduce il contesto che fa uso di una maschera **look-ahead** per mantenere la progressività dei dati.

Il contesto viene poi classificato per dedurne la velocità ideale alla quale l'agente cercherà di attenersi usando un `MLP`.


In [4]:
from source.AC import roboCar

agent = roboCar().to(torch.device('cuda'))

print(f"Trainable parameters: {sum(p.numel() for p in agent.parameters() if p.requires_grad)}")

Trainable parameters: 220817


## Pre training

L'addestramento avviene direttamente in simulazione, il modello fornisce un numero in $(0,1)$.

Questo numero indicherà la percentuale della $V_{\text{max}}$ che il veicolo vorrebbe seguire. In input saranno presi quindi solo $\delta_x\, , \, \delta_v$.

In parole povere il veicolo è addestrato per studiare il moto relativo con il veicolo di fronte e capirne le giuste reazioni per migliorare il traffico.

Poiché la strada è circolare, la guida negativa di un veicolo si riperquote su sé stesso in un futuro poiché gli altri veicoli pensano solo a sé stessi.

La rete quindi deve cercare di 'sistemare' il traffico.

### feedback per il guadagno

Il guadagno scelto è l'energia cinetica dei veicoli, che già si sa che tende a diminuire a causa delle code fantasma.

Il punto interessante è che l'energia cinetica può aumentare se il veicolo autonomo massimizza la sua velocità, tuttavia questo comportamento peggiora il traffico.

Paradossalmente se il veicolo autonomo massimizza l'energia cinetica totale pensando solo al suo addendo, sta peggiorando l'energia cinetica.

Il risultato atteso è che il veicolo autonomo cerchi di aumentare l'energia cinetica inducendo gli altri veicoli ad una guida corretta.

$$ \mathcal{G} = \sum_{n} \frac{1}{2}v_n^2 $$

Poiché si è in un modello dinamico, la rete eseguirà per tempo di simulazione $T$ la rete neurale e valuterà come feedback il guadagno di energia cinetica:

$$ \mathcal{G} = \sum_{n} \frac{1}{2}v_n(T)^2 $$

In [5]:
from source import training

# estrapolo un batch dal dataloader
training.train(
    agent,
    dataloader,
    optimizer=torch.optim.Adam(agent.parameters(), lr=1e-4, maximize=True),
    epochs=2,
    time_step = 1/30,
    steps = 3,
    deep_steps=600,
    with_reset=True)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Batch:   0%|          | 0/32 [00:00<?, ?it/s]

Simulation:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 19.94 MiB is free. Including non-PyTorch memory, this process has 3.52 GiB memory in use. Of the allocated memory 3.44 GiB is allocated by PyTorch, and 17.41 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Salvo il pretraining così da poterlo riaprire senza perderlo

In [ ]:
agent.save_weights('data/models/pre_trained.pth')